In [ ]:
from utils import certificate_generator
with open('students.csv', 'r') as f:
    for line in f:
        name, cert = line.strip().split(',')
        certificate_generator.generate_certificate(name, 'utils/template.png', f'gen_certs/{name}.png', cert)
        print(f'Certificate generated for {name}')

In [ ]:
from utils import sign_certificates
crt = sign_certificates.SecureDocumentVerifier("HTTP://127.0.0.1:7545","0xbc56bb97DCAe27474b4bBDc2186D671BBbEC0d32","0xc79fdec90c65923f5d999516261d318257d6cf0910877a8b57926f9aaa726b1c")

In [ ]:
import os
for cert in os.listdir('gen_certs'):
    cert_path = os.path.join('gen_certs', cert)
    with open(cert_path, 'rb') as f:
        cert_hash, cert_sign = crt.sign_message(f.read())
    with open("certificates.csv", "a") as f:
        name = cert.split('.')[0]
        f.write(f"{name},{cert_hash},{cert_sign}\n")
        print(f'Certificate signed for {name}')
 

In [ ]:
from utils import add_code_to_certi
add_code_to_certi.add_qr_code("certificates.csv", "gen_certs/", 'gen_certs_with_qr/')

In [ ]:
from solcx import compile_standard, install_solc
import json

install_solc("0.8.0")

with open("contract.sol", "r") as f:
    contract = f.read()

compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"contract.sol": {"content": contract}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.0",
)

with open("compiled_code.json", "w") as f:
    f.write(json.dumps(compiled_sol))

In [ ]:
with open("compiled_code.json", "r") as f:
    compiled_code = json.load(f)

bytecode = compiled_code["contracts"]["contract.sol"]["DocumentRegistry"]["evm"]["bytecode"]["object"]
abi = compiled_code["contracts"]["contract.sol"]["DocumentRegistry"]["abi"]

In [ ]:
receipt = crt.deploy_contract(bytecode,abi)
contract_address = receipt.contractAddress
print(f"Contract deployed at {contract_address}")

In [ ]:
with open("certificates.csv", "r") as f:
    for line in f:
        name, cert_hash, cert_sign = line.strip().split(',')
        crt.add_document(cert_hash, cert_sign, contract_address, abi)
        print(f"Document added successfully for {name}")

In [ ]:
crt.verify_document("0xa1f71e9ee31b3f22b1846d7824dde1226209590b85f1b58a26dba39390bad7af", contract_address, abi)

In [ ]:
malicious_account = sign_certificates.SecureDocumentVerifier("HTTP://127.0.0.1:7545","0xDE9CC0FD19ec8Ab5d54606A6E8f1cA9D2F54057b","0xe412e3dc507754be0e7328f643c53725031c6f203a71bbb9954c444a30d58149")
malicious_hash, malicious_sign = malicious_account.sign_message(open("contract.sol", "rb").read())
#malicious_account.add_document(malicious_hash, malicious_sign, contract_address, abi)
crt.add_document(malicious_hash, malicious_sign, contract_address, abi)

In [ ]:
import requests
with open("certificates.csv", "r") as f:
    for line in f:
        name, cert_hash, cert_sign = line.strip().split(',')
        response = requests.post("http://localhost:5000/upload",files={"image":(cert_hash+".png",open(f"gen_certs/{name}.png","rb"))})
        print(response.text)

In [ ]:
from utils import scan_qr_code
import cv2
from eth_account.messages import defunct_hash_message
import requests
from IPython.display import display, Image

cap = cv2.VideoCapture(0)
if not cap.isOpened():
  print("Error opening webcam")
code = scan_qr_code.scan_qr_code(cap)
print(code)
if crt.verify_document(code, contract_address, abi):
    print("Document verified successfully")
    response = requests.get(f"http://localhost:5000/download/{code}.png")
    if response.status_code == 200:
        img = response.content
        if defunct_hash_message(img).hex() == code:
            print("The downloaded image is correct")
            display(Image(data=img))
        else:
            print("The downloaded image is incorrect")
    else:
        print("Thank you")
else:
    print("Document verification failed")